# 1. Parser

In [2]:
!pip install parsimonious

  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42724 sha256=906b421b6b5643019c8cdc4bbd14a59647ec1e6cac86fb397e2391891907edea
  Stored in directory: c:\users\simon\appdata\local\pip\cache\wheels\ae\3f\87\24298980f6c0436680b8b64caac154f542e138487ed0f90f2a
Successfully built parsimonious


In [3]:
from aloe.parser import AloeParser
from aloe.clause import *
parser = AloeParser()

In [4]:
try:
    P = parser.parse_file("aloe/Examples/toparse.pl")
except Exception as e:
    t = e
    raise(e)

In [5]:
print(P)

Background:
personne(anne).
personne(marie).
personne(jean).
personne(jacques).
echec_pre(jean).
echec_pre(jacques).

Examples:
{'pos': [echec_cours(jean)., echec_cours(jacques).], 'neg': [echec_cours(anne)., echec_cours(marie).]}

Modes:

options:
c: 2
h: 30
verbose: 1


# 2. Aloe engine
## 2.1 Matching atoms together

205-208 handout

mortel(X) :- man(X).
man(socrates).

mortel(socrates)?
mortel(X)?

In [14]:
# When initializing background

from aloe.clause import *
class FunctionCollection:
    def __init__(self, functions, clauses):
        self.clauses = set(clauses)
        self.collec = dict()
        for func, clause in zip(functions, clauses):
            self.add(func, clause)
            
    def add(self, func, clause):
        self.clauses.add(clause)
        fname = func.longname
        if fname not in self.collec:
            self.collec[fname] = [{'Vars':set()} for _ in range(func.arity)]
        for term, term_dict in zip(func, self.collec[fname]):
            if isinstance(term, Constant):
                if not term.name in term_dict:
                    term_dict[term.name] = set()
                term_dict[term.name].add(clause)    
            elif isinstance(term, Variable):
                term_dict['Vars'].add(clause)                        
            elif isinstance(term, Function):
                if 'Funcs' not in term_dict:
                    term_dict['Funcs'] = FunctionCollection([],[])                       
                term_dict['Funcs'].add(term,clause)
            
    def match(self, expr):
        name = expr.longname
        if name not in self.collec:
            return set()
        sets = []
        for term, term_dict in zip(expr, self.collec[name]):
            if isinstance(term, Constant):
                if term.name in term_dict:
                    sets.append(term_dict[term.name] | term_dict['Vars'])
                else:
                    sets.append(term_dict['Vars'])
            elif isinstance(term, Variable):
                sets.append(self.clauses)
            elif isinstance(term, Function):
                sets.append(term_dict['Funcs'].match(term))
        return set.intersection(*sets)
            
    def __repr__(self):
        return repr(self.collec)

In [15]:
atoms = [parser.parse_rule('p(t(%s))'%s,'atom') for s in ['a','b','c','d','F']]
col = FunctionCollection(atoms, atoms)
query = parser.parse_rule('p(t(a))','atom')
col.match(query)

{p(t('F')), p(t(a))}

In [16]:
class VariableBank:
    def __init__(self):
        self.variables = set()
        self.count = 0
                    
    def newVariable(self):
        var = Variable(self.count)
        self.variables.add(var)
        self.count += 1
        return var
    
    @staticmethod
    def build_from_atom(atom):
        def update_variables(func):
            for term in func:
                if isinstance(term, Variable):
                    vb.variables.add(term)
                elif isinstance(term, Constant):
                    continue
                elif isinstance(term, Function):
                    update_variables(term)
        vb = VariableBank()
        update_variables(atom)
        return vb

    def transform_clause(self, clause, subst=None):
        if subst is None: subst = dict()
        head = self.transform_atom(clause.head, subst)
        body = [self.transform_atom(b, subst) for b in self.body]
        return Clause(head, body)
    
    def transform_atom(self, atom, subst=None):
        if subst is None: subst = dict()
        p_name = atom.name
        p_args = [self.transform_term(t, subst) for t in term]
        return Predicate(p_name, p_args)
    
    def transform_term(self, term, subst=None):
        if subst is None: subst = dict()
        if isinstance(term, Variable):
            if term in subst:
                return subst[term]
            else:
                var = newVariable()
                subst[term] = var
                return var
        elif isinstance(term, Constant):
            return Constant
        elif isinstance(term, Function):
            f_name = term.name
            f_args = [self.transform_term(t, subst) for t in term]
            return Function(f_name, f_args)
    

In [38]:
# aloe.query
# Matching 2 atoms
CST, VAR, FUN = Constant.__name__, Variable.__name__, Function.__name__
def match(atom1, atom2):    
    """
    Tries to match two atoms together and returns the corresponding substitution.
    If not possible, returns None.
    """
    classes = atom1.__class__, atom2.__class__
    if   classes == (CST, CST):
        if atom1 == atom2:
            return {}
        else:
            return None
    elif classes in [(FUN, CST), (CST, FUN)]:
        return None
    elif classes == (FUN, FUN):
        if atom1.name != atom2.name or atom1.arity != atom2.arity:
            return None
        else:
            subst = dict()
            terms1, terms2 = atom1.terms.copy(), atoms2.terms.copy()
            for term1, term2 in zip(terms1, terms2):
                s = match(term1, term2)
                if s is None:
                    return None
                for var, term in s:
                    if var in subst:
                        term1.append(term)
                        term2.append(subst[var])
                    else:
                        subst[var] = term
            return subst
    elif classes in [(VAR, CST), (VAR, FUN)]:
        return {atom1:atom2}
    elif classes in [(CST, VAR), (VAR, VAR), (FUN, VAR)]:
        return {atom2:atom1}

def apply_subst(term, subst):
    """ 
    Applies variable substitution 'subst' to term 'term'.
    Variables that are not in 'subst' are returned as they are.
    """
    term_class = term.__class__
    if   term_class == CST:
        return term
    elif term_class == VAR:
        if term in subst:
            return subst[term]
        else:
            return term
    elif term_class == FUN:
        f_name = term.name
        f_args = [apply_subst(t, subst) for t in f_args]
        return Function(f_name, f_args)
        
def var_in_term(var, term):
    """ Checks whether variable 'var' can be found in term 'term' """
    term_class = term.__class__
    if   term_class == CST:
        return False
    elif term_class == VAR:
        return term==var
    elif term_class == FUN:
        return any([var_in_term(var,t) for t in term])
    
def unify(subst):
    """ 
    Procedes to the unification of substitution. 
    In the case that such a substitution is not valid, None is returned. 
    """
    new_subst = dict()
    for var, term in subst:
        n_term = apply_subst(term, new_subst)
        if var_in_term(var, n_term): # Fail to unify
            return None
        new_subst[var] = n_term
    return new_subst


In [19]:
Constant.__name__

'Constant'

In [24]:
t = "b(cO,c,c(d),b(x))."
clause = parser.parse_rule(t ,"hornclause")
atom = clause.head

In [25]:
t = parser.parse_rule("a",'constant')
type(t)

vocabulary.Constant

In [8]:
clause.head.terms[0]

cO

In [9]:
f = FunctionCollection([atom],[clause])

<class 'vocabulary.Constant'>
<class 'vocabulary.Constant'>
<class 'vocabulary.Function'>
<class 'vocabulary.Constant'>
<class 'vocabulary.Function'>
<class 'vocabulary.Constant'>


In [10]:
f.collec

{'b/4': [{'Vars': set(), 'cO': {b(cO,c,c(d),b(x)).}},
  {'Vars': set(), 'c': {b(cO,c,c(d),b(x)).}},
  {'Vars': set(), 'Funcs': <__main__.FunctionCollection at 0x7f24984f5160>},
  {'Vars': set(), 'Funcs': <__main__.FunctionCollection at 0x7f2498478640>}]}

In [ ]:
class LogicProgram:
    def __init__(self, clauses=None):
        self.clauses = clauses if clauses else list()
        self.unit_pos = dict()
        self.unit_neg = dict()

In [ ]:
subst = {num:{Var:val}}

In [ ]:
atom
candidates = clauses[pred.name]
for 



In [3]:
def add_subst(term1, term2, subst1, subst2):
    if isinstance(term2, Variable):
        if term2 in subst2:
            old_subst = subst[term2]
            if old_subst == term1:
                return term1
            elif 
            if 
        subst[term2] = term1
        return term1

True

In [20]:
a = (1,2)
a == (_,2)

False

In [28]:
class A:
    def __init__(self):
        self.a = [2,3,4]
     
    def __iter__(self):
        return iter(self.a)

help(in)

SyntaxError: invalid syntax (<ipython-input-28-8a8245909101>, line 8)

In [13]:
a = {1:3, 2:4}
for i,j in a.items():
    print(i,j)

1 3
2 4


|
a - b - c (list of atoms), (var bank), ()

In [15]:
isinstance(a, (list, dict))

True

In [18]:
tuple(a)

(1, 2)

In [89]:
@dataclass        
class Options(object):
    def __init__(self, options=[]):
        # Default values for options
        self.c = 2
        self.h = 30
        self.verbose = 0
        
        # Here we introduce the user's options
        if isinstance(options, dict): 
            options = options.items()
        for attr, value in options:
            setattr(self, attr, value)
            
    def __repr__(self):
        return '\n'.join(['%s: %s' % (attr, str(getattr(self, attr))) for attr in self.__slots__])


In [90]:
help(dataclass)

Help on function dataclass in module dataclasses:

dataclass(cls=None, /, *, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
    Returns the same class as was passed in, with dunder methods
    added based on the fields defined in the class.
    
    Examines PEP 526 __annotations__ to determine fields.
    
    If init is true, an __init__() method is added to the class. If
    repr is true, a __repr__() method is added. If order is true, rich
    comparison dunder methods are added. If unsafe_hash is true, a
    __hash__() method function is added. If frozen is true, fields may
    not be assigned to after instance creation.



In [91]:
o = Options([('c',3)])

In [93]:
i=o.__class__()

In [80]:
o.f = 3
o.f

AttributeError: aloe.options.Options object has no 'f' attribute

In [85]:
isinstance([1],list(int))

TypeError: 'type' object is not iterable